![](https://www.qads.com.br/img/QADS.png)

#  Exercícios do módulo de Obtenção e Preparação de Dados

Remis Balaniuk, PhD

# Flat files

Obtenha os dados abertos das [viagens realizadas a serviço no Portal daTransparência](!http://www.portaltransparencia.gov.br/download-de-dados/viagens), processe as colunas normalizando atributos textuais e convertendo os demais para os tipos corretos e ao final salve o(s) dataframe(s) em disco usando o Pickle.

# Expressões regulares

Escreva as expressões regulares solicitadas. Cada exercício possui um conjunto de testes que deve ser completamente validado pela expressão regular. Cada teste possui registros onde expera-se um *match* da expressão regular, enquanto outros registros não devem apresentar nenhum *match*. Faça o exercício para que todos os registros do teste fiquem com a marca de correto (em verde). Para os exercícios, não utilize expressões regulares com mais de 80 caracteres.



Para esse exercício você precisará baixar o arquivo zipado "exerciciosRegex" disponibilizado na pasta QADS em (https://drive.google.com/drive/folders/1sUeC2VYx93ZXi3tKCJarUTVvq_7C2ECI?usp=sharing), descompactar esse arquivo e copiar todo o conteúdo em uma pasta do seu Google drive (contém um diretório chamado "exercicios" e um arquivo chamado re_jupyter.py). Em seguida você deve montar seu Google Drive aqui no Colab e acessar a pasta onde copiou esse conteúdo .

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive/QADS

/content/drive/MyDrive/QADS


In [ ]:
%pwd

'/content/drive/MyDrive/QADS'

In [ ]:
import imp
import re_jupyter
imp.reload(re_jupyter)

from re_jupyter import exercicio

Uma letra `a` imediatamente seguida por uma ou mais letras `b`

In [ ]:
exercicio(1, r'')

,ab
,abb
,abbbbbbbbb
,cbaaabb
,aba
,a
,baa
,ba
,cba
,cbaaa
,ba


Uma palavra de 6 letras que inicia com a letra `z`

In [ ]:
exercicio(2, r'')

,zangas
,zangas espaço
,zéfiro
,segundo zéfiro
,zelo
,zelos
,zangada
,zangados
,azul
,aziago
,azougue


Uma palavra com 8 a 10 letras que termine com a letra 'z'

In [ ]:
exercicio(3, r'')

,robustez
,espaço robustez
,chafariz
,antes chafariz depois
,vetustez
,placidez
,insensatez
,gravidez
,invalidez
,luz
,talvez


3 dígitos consecutivos

In [ ]:
exercicio(4, r'')

,040 inicia com zero
,056 de novo
,100 cem
,789 789 789
,0101 binário
,000x termina em x
,ano de 1984
,data 01/01/2018
,zero 0
,dígito 1
,vinte 20


Um número inteiro com 2, 3 ou 4 dígitos, sendo que o primeiro dígito não pode ser zero

In [ ]:
exercicio(5, r'')

,vinte 20
,100 cem
,789 789 789
,ano de 1984
,data 01/01/2018
,zero 0
,dígito 1
,inicia com zero 03
,03 também tem zero
,040 inicia com zero
,056 de novo


Uma palavra que inicie com um dígito e termine com um letra

In [ ]:
exercicio(6, r'')

,parágrafo 11b
,1111111b
,1bbbbbbb
,1b1b1b1b
,teste 1a
,dígitos 113 total
,soma quando 3a
,bb1bbbbb11b
,b1b1b1b1
,111
,1a1


Uma palavra seguida de um espaço seguido de um sequência de um ou mais dígitos

In [ ]:
exercicio(7, r'')

,teste 1234
,parágrafo 12345
,letra 1
,a b 1 c
,cc 11 a
,1 1
,dois espaços não pode 2


Um dia da semana: segunda-feira, terça-feira, quarta-feira, quinta-feira, sexta-feira. Adicionalmente, permita a escrita dos dias da semana com espaço no lugar do hífen ou com as palavras unidas (ex.: quartafeira e quinta feira).

In [ ]:
exercicio(8, r'')

,hoje é segunda-feira
,amanhã será terça feira
,quartafeira já passou
,quinta-feira foi dia 3
,sextafeira
,sábado feriado
,domingo feriado
,2a feira internacional
,sexta_feira
,quinta.feira
,domingues


Uma URL no formato: http://www.&lt;dominio&gt;.com.br/&lt;arquivo&gt;.html. O &lt;arquivo&gt; e o &lt;domínio&gt; somente podem contar letras de a a z (minúscula ou maiúsculas), dígitos e os sinais _ e -.

In [ ]:
exercicio(9, r'')

,http://www.google.com.br/index.html
,http://www.terra.com.br/underline_1.html
,http://www.teste-novo.com.br/hiFen-1.html
,http://www.google.com/index.html
,http://uol.com.br/teste.html
,http://bol.com.br/nao/podem/existir/barras.html
,https://www.google.com/https/também/nao/pode.html
,http://www.teste_velha.com.br/111-html
,http://www-testevelho.com-br/111.html
,http://www..com-br/1.html
,http://www.com-br/2.html


Duas palavras unidas pelas preposições 'de', 'do', 'da', 'dos' ou 'das'. As palavras somente podem conter letras e elas são separadas por um ou mais espaços.

In [ ]:
exercicio(10, r'')

,"Olhos do mundo,"
,"zelos do marido,"
,"morte do Viegas,"
,uma vertigem de cabriolas
,descendente do cavalo de Aquiles
,a minha teoria das edições humanas
,me pareceu entrar na região dos gelos eternos
,"Esta idéia,"
,"rútila e grande,"
,"trajada ao bizarro,"
,"como diria o Padre Bernardes,"


Uma palavra de no mínimo 3 letras e com TODAS as letras maiúsculas

In [ ]:
exercicio(11, r'')

,"ALBERTO YOUSSEF, brasileiro, casado, filho de Kalim Youssef e Antonieta Youssef"
,"PAULO ROBERTO COSTA, ex-Diretor de Abastecimento da PETROBRAS"
,"WALDOMIRO DE OLIVEIRA, conhecido “Bom Velhinho”"
,"JOSÉ ALDEMÁRIO PINHEIRO FILHO, vulgo “Léo Pinheiro”,"
,"AGENOR FRANKLIN MAGALHÃES MEDEIROS, brasileiro"
,empreiteiras ODEBRECHT
,integrantes do Grupo OAS
,empresa GFD INVESTIMENTOS
,conforme já apurado pelo TCU
,"no caso das empresas MO Consultoria, Empreiteira Rigidez e RCI Software"
,RELATÓRIO DE TENDÊNCIAs


Duas palavras consecutivas, de no mínimo 2 letras cada, com TODAS as letras maiúsculas

In [ ]:
exercicio(12, r'')

,"ALBERTO YOUSSEF, brasileiro, casado, filho de Kalim Youssef e Antonieta Youssef"
,"PAULO ROBERTO COSTA, ex-Diretor de Abastecimento da PETROBRAS"
,"WALDOMIRO DE OLIVEIRA, conhecido “Bom Velhinho”"
,"JOSÉ ALDEMÁRIO PINHEIRO FILHO, vulgo “Léo Pinheiro”,"
,"AGENOR FRANKLIN MAGALHÃES MEDEIROS, brasileiro"
,empresa GFD INVESTIMENTOS
,RELATÓRIO DE TENDÊNCIAs
,VARIOS ESPACOS
,empreiteiras ODEBRECHT
,integrantes do Grupo OAS
,conforme já apurado pelo TCU


Duas ou mais palavras consecutivas com todas as letras maiúsculas. Cada palavra pode possuir qualquer quantidade de letras.

In [ ]:
exercicio(13, r'')

,"ALBERTO YOUSSEF, brasileiro, casado, filho de Kalim Youssef e Antonieta Youssef"
,"PAULO ROBERTO COSTA, ex-Diretor de Abastecimento da PETROBRAS"
,"WALDOMIRO DE OLIVEIRA, conhecido “Bom Velhinho”"
,"JOSÉ ALDEMÁRIO PINHEIRO FILHO, vulgo “Léo Pinheiro”,"
,"AGENOR FRANKLIN MAGALHÃES MEDEIROS, brasileiro"
,empresa GFD INVESTIMENTOS
,RELATÓRIO DE TENDÊNCIAs
,TESTE A TESTE
,A BE C DE
,VARIOS ESPACOS
,empreiteiras ODEBRECHT


Uma linha contendo exatamente um endereço [IP](https://en.wikipedia.org/wiki/IP_address). ex.: 127.0.0.1, 192.168.0.254. Um IP é formado por exatamente 4 números separados por pontos. Cada número possui no máximo 3 dígitos.

In [ ]:
exercicio(14, r'')

,127.0.0.1
,192.168.0.254
,10.11.10.11
,127.127.127.127
,9999.0.0.1
,127404041
,a127.0.0.1
,127.a.a.1
,127.0.0.1a
,xxx.yyy.zzz.www
,0.0.0.a


Uma linha contendo exatamente um CPF com separador de milhares e hífen.

In [ ]:
exercicio(15, r'')

,000.000.000-00
,111.111.111-11
,0111.111.111-11
,111.111.111-110
,111.0111.0111-11
,11101110111-11
,111.111.111111
,1110110111111
,111011011111
,11101101111
,1110110111


Uma linha contendo exatamente um CPF com separador de milhares opcional e hífen opcional.

In [ ]:
exercicio(16, r'')

,000.000.000-00
,111.111.111-11
,111111.111-11
,111.111111-11
,11101101111
,0111.111.111-11
,111.111.111-110
,111.0111.0111-11
,11101110111-11
,111.111.111111
,1110110111111


Uma linha contendo exatamente um CNPJ com separador de milhares, barra e hífen.

In [ ]:
exercicio(17, r'')

,00.000.000/0001-00
,xx.000.000/0001-xx
,0011001000/0001-00
,00.000.00010001-00
,00.000.000/0001100
,00000.000/0001-00
,00000000/0001-00
,000000000001-00
,00000000000100
,x00000000000100
,00000000000100x


Uma linha contendo exatamente um CNPJ com separador de milhares opcional, barra opcional e hífen opcional.

In [ ]:
exercicio(18, r'')

,00.000.000/0001-00
,00000.000/0001-00
,00000000/0001-00
,000000000001-00
,00000000000100
,xx.000.000/0001-xx
,0011001000/0001-00
,00.000.00010001-00
,00.000.000/0001100
,x00000000000100
,00000000000100x


Uma linha contendo exatamente um CEP com separador de milhares e hífen

In [ ]:
exercicio(19, r'')

,70.000-000
,7a.aaa-aaa
,7000000000
,70000-000
,700000000
,70000000
,7000000
,700000
,70000
,7000
,70000aaa


Uma linha contendo exatamente um CEP com separador de milhares opcional e hífen opcional

In [ ]:
exercicio(20, r'')

,70.000-000
,70000-000
,70000000
,7a.aaa-aaa
,7000000000
,700000000
,7000000
,700000
,70000
,7000
,70000aaa


Uma linha contendo exatamente um telefone no formato (xx)xxxx-xxxx.

In [ ]:
exercicio(21, r'')

,(61)3316-0000
,3316-0000
,x61x3316x0000
,(61)3316x0000
,((61)3316-00006
,77(61)3316x000077
,(77)(61)3316-00007


Uma linha contendo exatamente um telefone no formato (xx)xxxx-xxxx ou xxxx-xxxx (sem DDD).

In [ ]:
exercicio(22, r'')

,(61)3316-0000
,3316-0000
,x61x3316x0000
,(61)3316x0000
,((61)3316-00006
,77(61)3316x000077
,(77)(61)3316-00007


# JSON

Usando a API do Banco Central, descrita em https://www.bcb.gov.br/conteudo/dadosabertos/BCBDepin/gnastportal-dados-abertostaxas-de-cambio---todos-os-boletins-diarios.pdf, escreva uma função que apresente ao usuário as moedas disponíveis para cotação, peça a ele que escolha uma ou mais, peça que o usuário escolha uma data inicial e uma data final, obtenha as cotações para as moedas selecionadas no período, armazene num dataframe e ao final salve o dataframe no disco usando o Pickle.

# HTML, JSON e REGEX

Crie um extrator de contratos a partir da seção 3 do Diário Oficial da União, obtendo as informações da página (link e texto) e por meio de expressões regulares extraia o nome do contratante, CNPJ e nome do contratado. Coloque o resultado num dataframe e salve em disco com o Pickle.